In [1]:
unmosaicked_input = 'out/DelawareRiverBasin/Run12_04_2020'
enduser_cog_output = 'enduser/DelawareRiverBasin/drb150/'
year='1954'
product='netet'
cmd_opt = '-i ' + unmosaicked_input + ' -o ' + enduser_cog_output + ' -y ' + year + ' ' + product + ' dummy'

NUM_CONTAINERS = 23 # 40 is too high, maybe 25

MAX_LOAD_LEVEL = 220
MIN_MEMORY_AVAILABLE = 4

MAX_CONCURRENT_CONTAINERS = NUM_CONTAINERS

start_year = 1950
#start_year = 2041
end_year = 2099

# Next Steps
1. make this an application so we can run it in tmux
2. add logging
3. add api - and product looping

In [2]:
import docker

In [3]:
# get client
client = docker.from_env()

In [4]:
running_containers = client.containers.list()
for c in running_containers:
    print(c.name)

happy_heisenberg


In [5]:
def start_container(client, docker_image, docker_full_cmd):
    container = client.containers.run(docker_image, docker_full_cmd, detach=True)
    print ( "CONTAINER is ", container.name)
    return(container)

In [6]:
def start_etm(year):
    global unmosaicked_input
    global enduser_cog_output
    global product
    
    cmd_opt = '-i ' + unmosaicked_input + ' -o ' + enduser_cog_output + ' -y ' + year + ' ' + product + ' dummy'
    print(cmd_opt)
    cmd = 'python3 api_etm.py '
    full_cmd = cmd + cmd_opt
    #print(full_cmd)
    docker_image =  "tbutzer/etm_docker_image"
    #print(docker_image)
    c = start_container(client, docker_image, full_cmd)
    print("real name is", c.name)
    print("==="*30)

In [7]:
for yeari in range(start_year, start_year+NUM_CONTAINERS+1):
    print(yeari)
    year=str(yeari)
    start_etm(year)

1950
-i out/DelawareRiverBasin/Run12_04_2020 -o enduser/DelawareRiverBasin/drb150/ -y 1950 netet dummy
CONTAINER is  bold_volhard
real name is bold_volhard
1951
-i out/DelawareRiverBasin/Run12_04_2020 -o enduser/DelawareRiverBasin/drb150/ -y 1951 netet dummy
CONTAINER is  kind_kirch
real name is kind_kirch
1952
-i out/DelawareRiverBasin/Run12_04_2020 -o enduser/DelawareRiverBasin/drb150/ -y 1952 netet dummy
CONTAINER is  interesting_maxwell
real name is interesting_maxwell
1953
-i out/DelawareRiverBasin/Run12_04_2020 -o enduser/DelawareRiverBasin/drb150/ -y 1953 netet dummy
CONTAINER is  stoic_matsumoto9
real name is stoic_matsumoto9
1954
-i out/DelawareRiverBasin/Run12_04_2020 -o enduser/DelawareRiverBasin/drb150/ -y 1954 netet dummy
CONTAINER is  serene_einstein
real name is serene_einstein
1955
-i out/DelawareRiverBasin/Run12_04_2020 -o enduser/DelawareRiverBasin/drb150/ -y 1955 netet dummy
CONTAINER is  tender_hawking1
real name is tender_hawking1
1956
-i out/DelawareRiverBasin/Run

In [8]:
running_containers = client.containers.list()
for c in running_containers:
    if len(c.attrs['Args']) >7 :
        print(c.name, c.attrs['Args'][6], c.attrs['Args'][7])

busy_lichterman 1973 netet
recursing_kare 1972 netet
trusting_khayyam 1971 netet
compassionate_kirch 1970 netet
thirsty_swanson 1969 netet
eager_driscoll 1968 netet
gifted_tharp 1967 netet
zealous_franklin 1966 netet
dazzling_bassi 1965 netet
sad_antonelli 1964 netet
practical_pare 1963 netet
wizardly_kepler 1962 netet
beautiful_bhabha 1961 netet
hungry_panini 1960 netet
sharp_leavitt 1959 netet
quizzical_kapitsa 1958 netet
hopeful_darwin 1957 netet
flamboyant_vaughan 1956 netet
tender_hawking1 1955 netet
serene_einstein 1954 netet
stoic_matsumoto9 1953 netet
interesting_maxwell 1952 netet
kind_kirch 1951 netet
bold_volhard 1950 netet


- https://medium.com/the-andela-way/machine-monitoring-tool-using-python-from-scratch-8d10411782fd

# Should maybe use os.getloadavg()[0]

In [9]:
import os
def return_cpu_load():
    cpu_load = [x / os.cpu_count() * 100 for x in os.getloadavg()][-1]
    return cpu_load

In [10]:
os.cpu_count()

8

In [11]:
import subprocess
import re

def _return_mem_stat():


    # Memory usage
    total_ram = subprocess.run(['free', '-h'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    used_free_shared_buf_avail = total_ram.split('\n')[1]
    
    return  used_free_shared_buf_avail

def return_available_memory():
    used_free_shared_buf_avail = _return_mem_stat()
    a = re.split('\s+', used_free_shared_buf_avail)
    available_memory = a[3].split('G')[0]
    return float(available_memory)

In [12]:
_return_mem_stat()

'Mem:           31Gi       2.2Gi        27Gi       2.0Mi       1.7Gi        28Gi'

In [13]:
available_memory = return_available_memory()
available_memory

27.0

In [14]:
os.getloadavg()

(1.66, 0.37, 0.13)

In [15]:
def return_num_containers():
    global client
    running_containers = client.containers.list()
    return(len(running_containers))

In [16]:
import time
def event_loop(year_to_process, end_year):
    
    
    while year_to_process <= end_year:
        time.sleep(30)

        mem_avail = return_available_memory()
        cpu_load = return_cpu_load()
        num_running_containers = return_num_containers()
        print(mem_avail, cpu_load, num_running_containers)

        cpu = False
        if (cpu_load < MAX_LOAD_LEVEL):
            print("CPU is FINE")
            cpu=True

        mem = False
        if mem_avail > MIN_MEMORY_AVAILABLE:
            mem = True

        containers = False
        if num_running_containers < MAX_CONCURRENT_CONTAINERS:
            containers = True
        

        if (mem and cpu and containers):
            print("OK to Launch")
            start_etm(str(year_to_process)) ### start mosaic container
            print("starting year", year_to_process)
            year_to_process = year_to_process + 1

    
    

In [ ]:
cyear = start_year + NUM_CONTAINERS + 1
event_loop(cyear, end_year)

21.0 8.375 25
CPU is FINE
19.0 11.625 25
CPU is FINE
20.0 15.75 25
CPU is FINE
22.0 20.25 25
CPU is FINE
21.0 27.500000000000004 25
CPU is FINE
20.0 32.5 25
CPU is FINE
19.0 36.375 25
CPU is FINE
17.0 37.875 25
CPU is FINE
15.0 39.625 25
CPU is FINE
17.0 42.875 25
CPU is FINE
20.0 46.75 25
CPU is FINE
23.0 51.625 25
CPU is FINE
22.0 57.49999999999999 25
CPU is FINE
17.0 62.375 25
CPU is FINE
17.0 63.125 25
CPU is FINE
